In [1]:
import os

import pandas as pd
from data_loader import label_str2int_dict

rationale_names = ['td','itc']
print(os.getcwd())
os.chdir('/home/lyq/DataSet/FakeNews/weibo_dataset')
print(os.getcwd())


/home/lyq/PycharmProjects/QwenVLRationaleGenerate/data/weibo
/home/lyq/DataSet/FakeNews/weibo_dataset


In [2]:
import ast

weibo_df = pd.read_csv('weibo.csv')
weibo_df = weibo_df.rename(columns={
    'id':'source_id',
    'text':'content',
})

def get_image_id(image_file_name_list):
    return ast.literal_eval(image_file_name_list)[0].split('/')[-1].split('.')[0]


weibo_df['content'] = weibo_df.apply(lambda x : f'{x["content"]} --发布来源：{x["release_source"]}', axis=1)
weibo_df['label'] = weibo_df['label'].apply(lambda x : label_str2int_dict[x])
weibo_df['image_id'] = weibo_df['available_image_paths'].apply(lambda x : get_image_id(x))

weibo_df = weibo_df.drop(columns=['available_image_paths','release_source'])

weibo_df

,source_id,content,split,label,image_id
0,3511947309647762,震惊，转发求证：【想都不敢想 ，在美国一桶金龙鱼食用油只要8元人民币】 一桶食用油相当于中国...,train,1,a71ac854gw1dytin2zmk9j
1,3576100079039606,【法院无底线】湖南长沙一位小朋友上学路上捡到3万元，原地不动等失主，结果被人冒领。不知情的孩...,train,1,3ba161e7jw1e4i6j4ep61j211n0r4juz
2,3899073935617462,"转发:我校需要小孩的衣服,新旧不限!西藏阿里地区是世界海拔最高的地区请问周围有没有四到十岁孩...",train,1,006ajvCngw1ex4jjl2pwlj30c8096wf7
3,3584521306131914,"立刻检查一下你家里的牙膏，如果是黑色条马上扔掉！ 大家买膏请留心,买牙膏时注意牙膏管反面 底...",train,1,6112c76ajw1e591p39mpej20hw094mxh
4,3553661986467439,快快转发，急急急！中央电视台《焦点访谈》已经播出，可口可乐承认旗下(果粒橙)含有美国禁用农药...,train,1,a5404163jw1e2iljwbrztj
...,...,...,...,...,...
7956,3896157270699471,【空城记：刘小东与鄂尔多斯】#正午故事#鄂尔多斯是个比较敏感的城市。其实政府也比想象的聪明，...,test,0,005EHRy1jw1ewv1azxwh1j30rs0lljzm
7957,3895644894378737,【日本人是怎么拍黑帮电影的？】Tim Gallo出生于俄国，17岁后迁往日本生活。通过用镜头...,test,0,005EHRy1jw1ewtlhknf30j30jg0czq4y
7958,3893257346088858,#跟着面叔看世界#第一站，日本东京筑地市场。你现在到哪儿了？不会还堵在路上吧？一起来晒晒旅途...,test,0,005EHRy1jw1ewlzj6oenaj30qo0zkdm5
7959,3892828281041275,【现场图·广西柳城县连续发生多起爆炸】据@南宁电视台 ：网友称，现场听到几声巨响，车站，市场...,test,0,61e04755jw1ewkm8o7fyyj20c80gb0u2


In [3]:
data_set_len = weibo_df.shape[0]

def findNoRationaleNews(df,r_df):
    return set(df['source_id'].tolist()) - set(r_df['source_id'].tolist())

for rationale_name in rationale_names:
    r_df = pd.read_csv(f'{rationale_name}.csv')
    r_df = r_df.dropna(subset=['authenticity','reason'])
    r_df['authenticity'] = r_df['authenticity'].apply(lambda x : label_str2int_dict[x])
    if r_df.shape[0] != data_set_len:
        raise Exception(f'{rationale_name} len : {r_df.shape[0]} != {data_set_len} , No Rationale id : {findNoRationaleNews(weibo_df,r_df)}')
    weibo_df = weibo_df.merge(r_df,how='left',on='source_id')
    weibo_df = weibo_df.rename(columns={
        'authenticity':f'{rationale_name}_pred',
        'reason': f'{rationale_name}_rationale'
    })
    weibo_df[f'{rationale_name}_acc'] = weibo_df.apply(lambda x : int(x[f'{rationale_name}_pred'] == x['label']), axis=1)


weibo_df

,source_id,content,split,label,image_id,td_pred,td_rationale,td_acc,itc_pred,itc_rationale,itc_acc
0,3511947309647762,震惊，转发求证：【想都不敢想 ，在美国一桶金龙鱼食用油只要8元人民币】 一桶食用油相当于中国...,train,1,a71ac854gw1dytin2zmk9j,1,1. **价格对比不合理**：该消息声称在美国沃尔玛，一桶金龙鱼食用油的价格低至1.3美元（...,1,1,新闻中提到的金龙鱼食用油在美国的价格与图中的金龙鱼食用油明显不符。新闻中提到的油是绿色纯天然...,1
1,3576100079039606,【法院无底线】湖南长沙一位小朋友上学路上捡到3万元，原地不动等失主，结果被人冒领。不知情的孩...,train,1,3ba161e7jw1e4i6j4ep61j211n0r4juz,1,1. **来源可信度**：该消息来源于个人微博账号“瀟湘墨人/@3千里”，并通过手机客户端发...,1,1,图片与新闻内容完全不相关。新闻报道的是关于小朋友捡到钱和法院判决的故事，而图片显示的是一把由...,1
2,3899073935617462,"转发:我校需要小孩的衣服,新旧不限!西藏阿里地区是世界海拔最高的地区请问周围有没有四到十岁孩...",train,1,006ajvCngw1ex4jjl2pwlj30c8096wf7,1,1. 发布来源：信息中提到的发布来源是“vivo X5Pro”，这是一个手机型号，而不是一个...,1,0,新闻报道中提到西藏阿里地区是世界海拔最高的地区，而图片中显示的正是高海拔地区的自然景观，包括...,0
3,3584521306131914,"立刻检查一下你家里的牙膏，如果是黑色条马上扔掉！ 大家买膏请留心,买牙膏时注意牙膏管反面 底...",train,1,6112c76ajw1e591p39mpej20hw094mxh,1,1. 该消息声称牙膏底部的颜色条代表不同的成分，但这种说法并没有科学依据或官方认证。\n2....,1,1,文中提到牙膏底部颜色条表示其成分，但图中只显示了牙膏的正面和背面，没有展示底部颜色条，无法验...,1
4,3553661986467439,快快转发，急急急！中央电视台《焦点访谈》已经播出，可口可乐承认旗下(果粒橙)含有美国禁用农药...,train,1,a5404163jw1e2iljwbrztj,1,\n1. **紧急呼吁**：开头的“快快转发，急急急！”是一种常见的谣言传播手法，目的是引起...,1,1,封面图片显示的是一朵荷花，与新闻内容中提到的可口可乐果粒橙和农药问题无关，无法提供任何关于新...,1
...,...,...,...,...,...,...,...,...,...,...,...
7956,3896157270699471,【空城记：刘小东与鄂尔多斯】#正午故事#鄂尔多斯是个比较敏感的城市。其实政府也比想象的聪明，...,test,0,005EHRy1jw1ewv1azxwh1j30rs0lljzm,0,1. 信息来源为微博，这是一个广泛使用的社交媒体平台，可以发布真实的信息。\n2. 信息中提...,1,0,图片显示的是一个城市建筑工地的场景，背景中可以看到高楼大厦，这与新闻摘要提到的鄂尔多斯作为一...,1
7957,3895644894378737,【日本人是怎么拍黑帮电影的？】Tim Gallo出生于俄国，17岁后迁往日本生活。通过用镜头...,test,0,005EHRy1jw1ewtlhknf30j30jg0czq4y,1,1. 消息中提到的Tim Gallo是一个不太为人知的名字，而且他的背景（出生于俄国，后来迁...,0,1,新闻报道提到的是关于日本黑帮电影的拍摄和摄影师Tim Gallo的工作，但提供的图片中是一个...,0
7958,3893257346088858,#跟着面叔看世界#第一站，日本东京筑地市场。你现在到哪儿了？不会还堵在路上吧？一起来晒晒旅途...,test,0,005EHRy1jw1ewlzj6oenaj30qo0zkdm5,1,1. 信息来源的可信度：信息来源未被指明，无法验证信息的真实性和可靠性。\n2. 内容性质：...,0,0,图片显示的是一个繁忙的市场场景，符合新闻中提到的日本东京筑地市场的描述。图片中的建筑和标识也...,1
7959,3892828281041275,【现场图·广西柳城县连续发生多起爆炸】据@南宁电视台 ：网友称，现场听到几声巨响，车站，市场...,test,0,61e04755jw1ewkm8o7fyyj20c80gb0u2,0,1. 信息来源：该消息来自微博上的南宁电视台，微博作为一个公开平台，其发布的信息通常可以追溯...,1,0,图片显示了城市中的建筑物和街道，烟雾升起，这与新闻报道中提到的“多起爆炸”和“车站、市场、医...,1


In [4]:

from Util import cal_rationale_metrics

for rationale_name in rationale_names:
    print(f'{rationale_name} metrics: \n '
          f'{cal_rationale_metrics(weibo_df[f"{rationale_name}_pred"].to_numpy(),weibo_df["label"].to_numpy())}')


td metrics: 
 {'acc': 0.8233890214797136, 'recall': 0.8141402388002694, 'recall_real': 0.7053816584294343, 'recall_fake': 0.9228988191711044, 'precision': 0.8424585086361885, 'precision_real': 0.8929440389294404, 'precision_fake': 0.7919729783429367, 'f1_macro': 0.8202978370390899, 'f1_real': 0.7881576928976837, 'f1_fake': 0.8524379811804962}
itc metrics: 
 {'acc': 0.8015324707951262, 'recall': 0.7873686299086613, 'recall_real': 0.6208127402526085, 'recall_fake': 0.953924519564714, 'precision': 0.8428454966033521, 'precision_real': 0.930835734870317, 'precision_fake': 0.754855258336387, 'f1_macro': 0.7938234455065132, 'f1_real': 0.7448525778290233, 'f1_fake': 0.8427943131840032}


In [5]:

from sklearn.model_selection import train_test_split


def split_train_val(df_train, val_size=0.2, random_state=666, stratify_column=None):
    """
    从训练集中划分出验证集。

    参数:
        df_train (pd.DataFrame): 输入的训练集 DataFrame。
        val_size (float): 验证集所占比例，默认为 0.2。
        random_state (int): 随机种子，确保结果可复现，默认为 666。
        stratify_column (str or None): 如果需要分层抽样，则提供类别列名；默认为 None。

    返回:
        tuple: 包含两个 DataFrame 的元组，分别为新的训练集和验证集。
    """
    if stratify_column and stratify_column not in df_train.columns:
        raise ValueError(f"列 '{stratify_column}' 不存在于提供的 DataFrame 中")

    stratify = df_train[stratify_column] if stratify_column else None

    df_new_train, df_val = train_test_split(
        df_train,
        test_size=val_size,
        random_state=random_state,
        stratify=stratify
    )

    return df_new_train, df_val

def count_dataset(df):
    return {
        'sum': df.shape[0],
        'real':(df['label']==label_str2int_dict['real']).sum(),
        'fake':(df['label']==label_str2int_dict['fake']).sum(),
    }

train_df,test_df = weibo_df[weibo_df['split']=='train'], weibo_df[weibo_df['split']=='test']
train_df,val_df = split_train_val(train_df)
print(f'train : {count_dataset(train_df)}')
print(f'val : {count_dataset(val_df)}')
print(f'test : {count_dataset(test_df)}')


train : {'sum': 4961, 'real': 2254, 'fake': 2707}
val : {'sum': 1241, 'real': 553, 'fake': 688}
test : {'sum': 1759, 'real': 835, 'fake': 924}


In [6]:
weibo_df.to_csv('weibo_llm_rationale.csv',index=False)
train_df.to_csv('train.csv',index=False)
val_df.to_csv('val.csv',index=False)
test_df.to_csv('test.csv',index=False)